In [18]:
import pickle
import os, sys
import subprocess
import pandas as pd
import streamlit as st
import matplotlib.pyplot as plt
import plotly.graph_objects as go

os.chdir('..')

from modules.eval import load,forces_load,eval_report

In [ ]:
def load_module(path,prop):
    dft,ml = load(f'{path}/dft_{prop}.pkl'),load(f'{path}/predicted_{prop}.pkl')
    return [ml,dft]

def prop_dict(structures,prop):

    values_dict,metrics_dict = {},{}

    for struc,dft,ml in zip(structures,prop[0],prop[1]):
        if struc[0] not in values_dict:
            values_dict[struc[0]] = [[dft],[ml]]
        else:
            values_dict[struc[0]][0].append(dft)
            values_dict[struc[0]][1].append(ml)
    
    for key,value in values_dict.items():

        e_r2,e_mae,e_rmse = eval_report(value[0],value[1])
        metrics_dict[key] = [e_r2,e_mae,e_rmse]
    
    return metrics_dict

In [11]:
path = '/home/p.zanineli/work/inference/experiments/zirconium-dioxide/results'
mlips = os.listdir(path)

if './ipycheckpoints' in mlips: mlips.remove('./ipycheckpoints')
if 'metrics.csv' in mlips: mlips.remove('metrics.csv')
if 'emissions' in mlips: mlips.remove('emissions')

In [34]:
for mlip in mlips:
    models = os.listdir(f'{path}/{mlip}')
    
    for model in models:
        
        model_path = f'{path}/{mlip}/{model}'
        
        structures = load(f'{model_path}/structures.pkl')
        energies = load_module(model_path,'energies')
        forces = load_module(model_path,'forces')

        force_modules = []
        for force in forces:
            force_module,x,y,z = forces_load(force)
            force_modules.append(force_module)
        
        e_geometries = prop_dict(structures,energies)
        f_geometries = prop_dict(structures,force_modules)

        break

In [35]:
e_metrics_geometries = {}

In [36]:
for key in e_geometries.keys():
    e_metrics_geometries[model] = e_geometries

In [74]:
e_metrics_geometries[model]

{'p': [0.9842830485, 0.07515504283570211, 0.08005253870606151],
 'b': [-0.1134997669, 0.07899475946158897, 0.07925230217862105],
 's': [-0.8671123985, 0.09816573832292504, 0.09866282049859178],
 'n': [-29.3799284643, 0.11897236747204766, 0.1190138949510587],
 'w': [0.7697538829, 0.09074019267812453, 0.09603717762272344]}

In [85]:
di = set_final_metrics(e_metrics_geometries)

In [95]:
['Model'] + geometries

['Model', 'p', 'b', 's', 'n', 'w']

In [104]:
keys = list(e_metrics_geometries.keys())
geometries = list(e_metrics_geometries[keys[0]].keys())

df = pd.DataFrame(di).T
df.columns = geometries